
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 確率と統計

:ラベル: `sec_prob`

いずれにせよ、機械学習には不確実性がつきものです。教師あり学習では、既知のもの (*特徴*) を与えて、未知のもの (*ターゲット*) を予測したいと考えます。目的に応じて、ターゲットの最も可能性の高い値を予測しようとする場合があります。あるいは、ターゲットからの予想距離が最小となる値を予測することもできます。また、特定の値を予測するだけでなく、*不確実性を定量化し*たい場合もあります。たとえば、患者を説明するいくつかの特徴が与えられた場合、その患者が来年心臓発作を起こす*可能性がどのくらいあるのか*を知りたいと思うかもしれません。教師なし学習では、不確実性が気になることがよくあります。一連の測定値が異常かどうかを判断するには、対象の母集団で値が観察される可能性がどの程度かを知ることが役立ちます。さらに、強化学習では、さまざまな環境で知的に動作するエージェントを開発したいと考えています。これには、環境がどのように変化すると予想されるか、および利用可能な各アクションに応じてどのような報酬が得られると予想されるかについて推論する必要があります。

*確率は*、不確実性の下での推論に関係する数学分野です。何らかのプロセスの確率モデルが与えられると、さまざまなイベントの可能性について推論することができます。反復可能なイベント (コイン投げなど) の頻度を説明するために確率を使用することには、ほとんど議論の余地がありません。実際、*頻度主義の*学者は、そのような反復可能な出来事に*のみ*適用される確率の解釈に固執しています。対照的に、*ベイジアンの*学者は、不確実性の下での推論を形式化するために確率という言語をより広範囲に使用します。ベイズ確率は、次の 2 つの独自の特徴によって特徴付けられます。(i) 再現不可能な出来事に信念の度合いを割り当てる。たとえば、月がチーズでできている*確率*はいくらか。 (ii) 主観性 --- ベイズ確率は、新しい証拠に照らして自分の信念を更新する方法について明確なルールを提供しますが、異なる個人が異なる*以前の*信念から始めることを可能にします。*統計は*、データの収集と整理から始めて、データを生成したプロセスについてどのような推論を導き出すかという、逆方向の推論に役立ちます。データセットを分析して、より広範な母集団を特徴付けると思われるパターンを探すときは常に、統計的思考を採用しています。ほとんどのコース、専攻、論文、キャリア、学部、企業、機関は、確率と統計の研究に専念してきました。このセクションは表面をなぞっただけですが、モデルの構築を開始するために必要な基礎を提供します。


In [1]:
%matplotlib inline
import random
import torch
from torch.distributions.multinomial import Multinomial
from d2l import torch as d2l


## 簡単な例: コインを投げる

コインを投げることを計画しており、表が出る可能性 (または裏が出る可能性) を定量化したいと考えていると想像してください。コインが*公平*な場合、両方の結果 (表と裏) の可能性が等しくなります。さらに、コインを $n$ 回投げることを計画している場合、*予想される*表の割合は、*予想*される裏の割合と正確に一致する必要があります。これを直感的に確認する方法の 1 つは、対称性によるものです。$n_h$ の表と $n_t = (n - n_h)$ の裏で考えられるすべての結果について、$n_t$ の表と $n_h$ の裏で同様に起こり得る結果が存在します。これは、平均して $1/2$ のトスが表になり、$1/2$ が裏になると予想される場合にのみ可能であることに注意してください。もちろん、$n=1000000$ ずつ投げてこの実験を何度も行った場合、$n_h = n_t$ が正確に一致する試行は決して見られないかもしれません。

正式には、$1/2$ という量は*確率*と呼ばれ、ここでは与えられたトスが表になる確率を表します。確率は、*イベント*と呼ばれる関心のある結果に $0$ から $1$ の間のスコアを割り当てます。ここで、関心のあるイベントは $\textrm{heads}$ であり、対応する確率 $P(\textrm{heads})$ を表します。 $1$ の確率は絶対的な確実性 (両側が表だったトリック コインを想像してください) を示し、$0$ の確率は不可能 (たとえば、両側が裏だった場合) を示します。頻度 $n_h/n$ と $n_t/n$ は確率ではなく*統計*です。確率は、データ生成プロセスの基礎となる*理論上の*量です。ここで、確率 $1/2$ はコイン自体のプロパティです。対照的に、統計は、観察されたデータの関数として計算される*経験的な*量です。確率的量と統計的量に対する私たちの関心は密接に絡み合っています。私たちは多くの場合、データセットを指定して確率などのモデル パラメーターの*推定値*を生成する*推定器*と呼ばれる特別な統計を設計します。さらに、これらの推定量が*一貫性*と呼ばれる優れた特性を満たす場合、推定値は対応する確率に収束します。さらに、これらの推定された確率は、将来遭遇する可能性のある同じ母集団からのデータの統計的特性を示します。

真の $P(\textrm{heads})$ が分からない本物のコインを偶然見つけたとします。統計的手法でこの量を調査するには、(i) いくつかのデータを収集する必要があります。 (ii) 推定器を設計する。ここでのデータ取得は簡単です。コインを何度も投げて、結果をすべて記録することができます。正式には、基礎となるランダムなプロセスから実現を引き出すことは、*サンプリング*と呼ばれます。ご想像のとおり、自然推定量の 1 つは、観察された*表*の数とトスの総数の間の割合です。

ここで、コインが実際に公正、つまり $P(\textrm{heads}) = 0.5$ だったと仮定します。公正なコインのトスをシミュレートするために、任意の乱数ジェネレーターを呼び出すことができます。確率 $0.5$ のイベントのサンプルを抽出する簡単な方法。たとえば、Python の`random.random`は区間 $[0,1]$ の数値を生成します。ここで、サブ区間 $[a, b] \subset [0,1]$ に含まれる確率は $ba$ に等しくなります。したがって、返された浮動小数点数が`0.5`より大きいかどうかをテストすることで、それぞれ`0.5`確率で`0`と`1`を取得できます。


In [2]:
num_tosses = 100
heads = sum([random.random() > 0.5 for _ in range(100)])
tails = num_tosses - heads
print("heads, tails: ", [heads, tails])

heads, tails:  [48, 52]



より一般的には、多項関数を呼び出し、最初の引数をドローの数に設定し、2 番目の引数を考えられるそれぞれの結果に関連付けられた確率のリスト。公正なコインの 10 回のトスをシミュレートするには、確率ベクトル`[0.5, 0.5]`を割り当て、インデックス 0 を表、インデックス 1 を裏と解釈します。この関数は、考えられる結果の数 (ここでは 2) に等しい長さのベクトルを返します。最初のコンポーネントは表の出現数を示し、2 番目のコンポーネントは裏の出現数を示します。


In [3]:
fair_probs = torch.tensor([0.5, 0.5])
Multinomial(100, fair_probs).sample()

tensor([44., 56.])


このサンプリング プロセスを実行するたびに、前回の結果とは異なる可能性がある新しいランダム値を受け取ります。トスの数で割ると、データ内の各結果の*頻度*が得られます。これらの頻度は、推定対象の確率と同様に、合計すると $1$ になることに注意してください。


In [4]:
Multinomial(100, fair_probs).sample() / 100

tensor([0.5300, 0.4700])


ここで、シミュレートされたコインが公平であるとしても (確率`[0.5, 0.5]`を自分たちで設定しました)、表と裏の数は同一ではない可能性があります。それは、限られた数のサンプルしか抽出できなかったためです。自分たちでシミュレーションを実装せず、結果だけを見た場合、コインがわずかに不公平であるかどうか、または $1/2$ からの逸脱の可能性がサンプルサイズが小さいことによる単なるアーチファクトであるかどうかをどのようにして知ることができるでしょうか? `10000`トスをシミュレーションするとどうなるかを見てみましょう。


In [5]:
counts = Multinomial(10000, fair_probs).sample()
counts / 10000

tensor([0.4970, 0.5030])


一般に、繰り返されるイベント (コイン投げなど) の平均の場合、繰り返し回数が増加するにつれて、推定値は真の基礎となる確率に収束することが保証されます。この現象の数学的証明は*大数の法則*と呼ばれ、*中心極限定理により*、多くの状況でサンプル サイズ $n$ が大きくなるにつれて、これらの誤差は $(1/\sqrt{ n})$。トスの数が`1`から`10000`に増加するにつれて推定値がどのように変化するかを研究して、さらに直観を深めてみましょう。


In [ ]:
counts = Multinomial(1, fair_probs).sample((10000,))
cum_counts = counts.cumsum(dim=0)
estimates = cum_counts / cum_counts.sum(dim=1, keepdims=True)
estimates = estimates.numpy()

d2l.set_figsize((4.5, 3.5))
d2l.plt.plot(estimates[:, 0], label=("P(coin=heads)"))
d2l.plt.plot(estimates[:, 1], label=("P(coin=tails)"))
d2l.plt.axhline(y=0.5, color='black', linestyle='dashed')
d2l.plt.gca().set_xlabel('Samples')
d2l.plt.gca().set_ylabel('Estimated probability')
d2l.plt.legend();


各実線の曲線は、コインの 2 つの値の 1 つに対応し、各グループの実験後にコインがその値になる推定確率を示します。黒い破線は、真の潜在的な確率を示します。より多くの実験を行ってより多くのデータを取得すると、曲線は真の確率に向かって収束します。あなたはすでに、統計学者を悩ませているいくつかのより高度な質問の形を理解し始めているかもしれません: この収束はどれくらいの速さで起こるのでしょうか?同じ工場で製造された多くのコインをすでにテストした場合、この情報をどのように組み込むことができるでしょうか?

## より正式な治療

私たちはすでに、確率モデルの提示、合成データの生成、統計的推定の実行、収束の経験的評価、誤差メトリクスの報告 (偏差のチェック) まで、かなりの作業を終えています。ただし、さらに先に進むには、より正確にする必要があります。

ランダム性を扱う場合、考えられる結果の集合 $\mathcal{S}$ を表し、それを*サンプル空間*または*結果空間*と呼びます。ここで、各要素は個別の可能な*結果*です。 1 枚のコインを転がす場合、$\mathcal{S} = {\textrm{表}, \textrm{裏}}$ となります。 1 つのサイコロの場合、$\mathcal{S} = {1, 2, 3, 4, 5, 6}$ となります。 2 枚のコインを投げた場合、考えられる結果は ${(\textrm{表}, \textrm{表})、(\textrm{表}, \textrm{裏})、(\textrm{裏}, \textrm{表}です)、(\textrm{尾}, \textrm{尾})}$。*イベントは*サンプル空間のサブセットです。たとえば、「最初のコイン投げで表が出た」というイベントは、集合 ${(\textrm{heads}, \textrm{heads}), (\textrm{heads}, \textrm{tails})}$ に対応します。ランダムな実験の結果 $z$ が $z \in \mathcal{A}$ を満たすときはいつでも、イベント $\mathcal{A}$ が発生します。サイコロの 1 つのロールについて、「$5$ が見える」イベント ($\mathcal{A} = {5}$) と「奇数が見える」イベント ($\mathcal{B} = {1, 3, 5}$)。この場合、サイコロの目が`5`なった場合、 $A$ と $B$ の両方が発生したと言えます。一方、$z = 3$ の場合、$\mathcal{A}$ は発生しませんが、$\mathcal{B}$ は発生します。

*確率*関数は、イベントを実数値 ${P: \mathcal{A} \subseteq \mathcal{S} \rightarrow [0,1]}$ にマッピングします。与えられたサンプル空間 $\mathcal{S}$ におけるイベント $\mathcal{A}$ の確率 ($P(\mathcal{A})$ で示される) は、次の特性を満たします。
- あらゆる事象の確率 $\mathcal{A}$ は非負の実数、つまり $P(\mathcal{A}) \geq 0$ です。
- サンプル空間全体の確率は $1$、つまり $P(\mathcal{S}) = 1$ です。
-  **相互に排他的*なイベント $\mathcal{A} 1、\mathcal{A}_2、\ldots$ の可算シーケンスに対して ($\mathcal{A}_i \cap \mathcal{A}_j = \emptyset$すべての $i \neq j$) について、それらのいずれかが起こる確率は、個々の確率の合計、つまり $P(\bigcup {i=1}^{\infty} \mathcal{A}* *i ) = \sum* {i=1}^{\infty} P(\mathcal{A}_i)$。

 :citet: `Kolmogorov.1933`によって提案されたこれらの確率論の公理は、多くの重要な結果を迅速に導き出すために適用できます。たとえば、あらゆる事象 $\mathcal{A}$*または*その補数 $\mathcal{A}&#39;$ が発生する確率は 1 であることがすぐにわかります (なぜなら $\mathcal{A} \cup \mathcal{A}&#39; = \mathcal{S}$)。 $1 = P(\mathcal{S} \cup \mathcal{S}&#39;) = P(\mathcal{S} \cup \emptyset) = P(\ であるため、$P(\emptyset) = 0$ であることも証明できます。 mathcal{S}) + P(\emptyset) = 1 + P(\emptyset)$。したがって、あらゆる事象 $\mathcal{A}$*と*その補数 $\mathcal{A}&#39;$ が同時に発生する確率は $P(\mathcal{A} \cap \mathcal{A}&#39;) = 0$ となります。非公式には、これは不可能な出来事が発生する確率がゼロであることを示しています。



## ランダム変数

サイコロを振ってオッズが出たり、最初のコイン投げで表が出たりといったイベントについて話すとき、私たちは*確率変数*のアイデアを持ち出していました。正式には、確率変数は、基礎となるサンプル空間から一連の (おそらく多数の) 値へのマッピングです。確率変数はサンプル空間とどのように異なるのか疑問に思うかもしれません。どちらも結果の集合であるからです。重要なのは、確率変数は生のサンプル空間よりもはるかに粗くなる可能性があることです。 $0$ と $1$ の間の線分など、基礎となるサンプル空間が無限である場合でも、「0.5 より大きい」などのバイナリ確率変数を定義できます。さらに、複数の確率変数が同じ基礎となるサンプル空間を共有できます。たとえば、「家の警報器が鳴るかどうか」と「私の家が強盗に遭ったかどうか」はどちらも、基礎となるサンプル空間を共有するバイナリ確率変数です。したがって、1 つの確率変数が取得する値を知ることで、別の確率変数の可能性の高い値について何かを知ることができます。警報機が鳴ったことを知ると、その家が強盗に遭ったのではないかと疑うかもしれません。

確率変数によって取得されるすべての値は、基礎となるサンプル空間のサブセットに対応します。したがって、確率変数 $X$ が値 $v$ をとる出来事 ($X=v$ で示される) は*イベント*であり、$P(X=v)$ はその確率を示します。場合によっては、この表記が不格好になることがあり、文脈が明確な場合に表記を乱用する可能性があります。たとえば、$P(X)$ を使用して、$X$ の*分布*、つまり $X$ が任意の値を取る確率を示す関数を広く参照することができます。また、確率変数 $X$ と $Y$ が表すすべての値に対して真であるステートメントを表現するための省略表現として、$P(X,Y) = P(X) P(Y)$ のような式を書くこともあります。つまり、すべての $i,j$ に対して、 $P(X=i \textrm{ and } Y=j) = P(X=i)P(Y=j)$ が成り立ちます。また、確率変数がコンテキストから明らかな場合に $P(v)$ と書いて表記法を悪用することもあります。確率論におけるイベントはサンプル空間からの結果のセットであるため、ランダム変数が取得する値の範囲を指定できます。たとえば、$P(1 \leq X \leq 3)$ は、イベント ${1 \leq X \leq 3}$ の確率を表します。

コインの投げやサイコロの投げのような*離散的*な確率変数と、母集団からランダムに抽出された人の体重や身長のような*連続的な*確率変数の間には微妙な違いがあることに注意してください。この場合、私たちは誰かの正確な身長を実際に気にすることはほとんどありません。さらに、十分に正確な測定を行った場合、地球上にまったく同じ身長を持つ人は 2 人もいないことがわかります。実際、十分に細かく測定すれば、起床時と就寝時の身長が同じになることはありません。誰かの身長が 1.801392782910287192 メートルである正確な確率について尋ねるのはほとんど意味がありません。その代わりに、私たちは通常、誰かの身長が特定の範囲、たとえば 1.79 メートルから 1.81 メートルの間にあるかどうかを判断できることを重視します。このような場合、確率*密度*を使用します。正確に 1.80 メートルの高さには確率はありませんが、密度はゼロではありません。区間に割り当てられた確率を取得するには、その区間にわたる密度の*積分*を取得する必要があります。



## 複数の確率変数

お気づきかもしれませんが、複数の確率変数間の相互作用を伴うステートメントを作成せずに最後のセクションを通過することさえできませんでした ($P(X,Y) = P(X) P(Y)$ を思い出してください)。機械学習のほとんどは、このような関係に関係しています。ここで、サンプル空間は対象となる母集団、たとえば企業と取引する顧客、インターネット上の写真、生物学者に知られているタンパク質などです。各確率変数は、異なる属性の (未知の) 値を表します。母集団から個人をサンプリングするときは常に、各確率変数の実現を観察します。確率変数によって取得される値は、重複している、部分的に重複している、または完全にバラバラである可能性があるサンプル空間のサブセットに対応するため、1 つの確率変数によって取得される値を知ることで、別の確率変数の値がどのような可能性が高いかについての信念を更新することができます。 。患者が病院に入ってきて、呼吸困難があり、嗅覚を失っていることが観察された場合、呼吸困難がなく、まったく普通の状態にある場合よりも、患者が新型コロナウイルス感染症に感染している可能性が高いと考えられます。嗅覚。

複数の確率変数を扱う場合、変数が共同で取り得る値のあらゆる組み合わせに対応するイベントを構築できます。これらの組み合わせ ($A=a$ や $B=b$ など) のそれぞれに確率を割り当てる*確率関数は結合確率*関数と呼ばれ、サンプル空間の対応するサブセットの共通部分に割り当てられた確率を単純に返します。*確率*変数 $A$ と $B$ がそれぞれ値 $a$ と $b$ を取るイベントに割り当てられる同時確率は $P(A = a, B = b)$ と表され、カンマは "and" を示します。 」。任意の値 $a$ および $b$ について、$P(A=a, B=b) \leq P(A=a)$ および $P(A=a, B=b) \leq が成立することに注意してください。 P(B = b)$、というのは、$A=a$ と $B=b$ が起こるためには、$A=a$ が起こらなければなら*ず、* $B=b$ も起こらなければならないからです。興味深いことに、同時確率は、これらの確率変数について確率的な意味で知ることができることをすべて示しており、個別の分布 $P(A)$ と $P(B)$ の回復など、他の多くの有用な量を導き出すために使用できます。 。 $P(A=a)$ を回復するには、確率変数 $B$ が取り得るすべての値 $v$ にわたって $P(A=a, B=v)$ を単純に合計します。 $P(A=a) = \sum_v P(A=a, B=v)$。

比率 $\frac{P(A=a, B=b)}{P(A=a)} \leq 1$ が非常に重要であることがわかります。これは*条件付き確率*と呼ばれ、「$\mid$」記号 $P(B=b \mid A=a) = P(A=a,B=b)/P(A=a) で表されます。 $。 $A=a$ が起こったという事実を条件にすると、イベント $B=b$ に関連する新しい確率がわかります。この条件付き確率は、$A=a$ に関連付けられたサンプル空間のサブセットのみに注目を制限し、すべての確率の合計が 1 になるように繰り込みを行うものと考えることができます。条件付き確率は実際には確率であり、すべての公理を尊重します。すべての項を同じイベントで条件付けし、注意を同じサンプル空間に限定する限り。たとえば、素のイベント $\mathcal{B}$ と $\mathcal{B}&#39;$ の場合、 $P(\mathcal{B} \cup \mathcal{B}&#39; \mid A = a) = P となります。 (\mathcal{B} \mid A = a) + P(\mathcal{B}&#39; \mid A = a)$。

条件付き確率の定義を使用すると、*ベイズの定理*と呼ばれる有名な結果を導き出すことができます。構造上、$P(A, B) = P(B\mid A) P(A)$ および $P(A, B) = P(A\mid B) P(B)$ となります。両方の方程式を組み合わせると、$P(B\mid A) P(A) = P(A\mid B) P(B)$ が得られます。

 $$P(A \mid B) = \frac{P(B\mid A) P(A)}{P(B)}.$$

この単純な方程式は、条件付けの順序を逆にすることができるため、深い意味を持ちます。 $P(B\mid A)$、$P(A)$、$P(B)$ を推定する方法がわかっていれば、$P(A\mid B)$ を推定できます。一方の項を直接推定するのは簡単でも、もう一方の項は推定できないことがよくありますが、ここではベイズの定理が役に立ちます。たとえば、特定の病気の症状の有病率、および病気と症状の全体的な有病率がそれぞれ分かれば、症状に基づいてその病気に罹患する可能性がどの程度かを判断できます。場合によっては、症状の蔓延など、$P(B)$ に直接アクセスできない場合があります。この場合、ベイズの定理の簡略版が役に立ちます。

 $$P(A \mid B) \propto P(B \mid A) P(A).$$

 $P(A \mid B)$ は $1$ に正規化する必要がある、つまり $\sum_a P(A=a \mid B) = 1$ であることがわかっているので、これを使って計算できます。

 $$P(A \mid B) = \frac{P(B \mid A) P(A)}{\sum_a P(B \mid A=a) P(A = a)}.$$

ベイズ統計では、観察者は、*事前の*$P(H)$ にエンコードされた利用可能な仮説のもっともらしさについての (主観的な) 事前信念と、次の値を観測する可能性がどの程度であるかを示す*尤度関数*を所有していると考えられます。クラス $P(E \mid H)$ 内の各仮説について収集された証拠。ベイズの定理は、利用可能な証拠 $E$ に照らして初期*事前*$P(H)$ を更新して*事後*信念 $P(H \mid E) = \frac{P(E \ミッド H) P(H)}{P(E)}$。非公式には、これは「事後確率は証拠で割った前の時間の尤度に等しい」と言えます。ここで、証拠 $P(E)$ はすべての仮説で同じであるため、単に仮説を正規化するだけで済みます。

 $\sum_a P(A=a \mid B) = 1$ を使用すると、確率変数を*周辺化する*こともできることに注意してください。つまり、$P(A, B)$ などの結合分布から変数を削除できます。結局のところ、私たちはそれを持っています

$$\sum_a P(B \mid A=a) P(A=a) = \sum_a P(B, A=a) = P(B).$$

独立性は、統計における多くの重要な考え方のバックボーンを形成するもう 1 つの基本的に重要な概念です。つまり、$A$ の値の条件付けによって $B$ に関連付けられた確率分布に変化が生じない場合、およびその逆の場合、2 つの変数は*独立*しています。より正式には、$A \perp B$ で示される独立性には、$P(A \mid B) = P(A)$ が必要であり、その結果、$P(A,B) = P(A \mid B) P が必要となります。 (B) = P(A) P(B)$。多くの場合、独立性は適切な前提となります。たとえば、確率変数 $A$ が 1 つの公正なコインを投げた結果を表し、確率変数 $B$ が別のコインを投げた結果を表す場合、$A$ が表になったかどうかを知ることは、$B$ の確率に影響を与えるべきではありません。頭が上がってくる。

独立性は、基礎となる分布からデータを連続的に抽出する際に独立性が維持される場合 (強力な統計的結論を下すことが可能)、またはデータ内のさまざまな変数間で独立性が維持される場合に特に役立ち、この独立性構造をエンコードする単純なモデルを使用できるようになります。 。一方、確率変数間の依存関係を推定することは、多くの場合、まさに学習の目的です。私たちは、病気と症状は独立して*いない*と考えているため、特に症状があれば病気の確率を推定することに重点を置いています。

条件付き確率は固有確率であるため、独立性と依存性の概念も適用されることに注意してください。 2 つの確率変数 $A$ と $B$ は、$P(A, B \mid C) = P(A \mid C)P(B \mid C)$ の場合に限り、3 番目の変数 $C$ が与えられた場合に*条件付きで独立し*ます。 。興味深いことに、2 つの変数は一般に独立している可能性がありますが、3 番目の変数で条件付けを行うと依存するようになります。これは、2 つの確率変数 $A$ と $B$ が 3 番目の変数 $C$ の原因に対応する場合によく発生します。たとえば、一般集団では骨折と肺がんは独立している可能性がありますが、入院するという条件を付けた場合、骨折と肺がんには負の相関があることが判明する可能性があります。それは、骨折によってその人が入院する理由*が説明され*、肺がんの可能性が低くなるからです。

逆に、2 つの従属確率変数は、3 つ目の確率変数を条件付けすると独立する可能性があります。これは、無関係な 2 つのイベントに共通の原因がある場合によく発生します。靴のサイズと読書レベルは小学生では高い相関があるが、年齢を条件にするとこの相関はなくなる。



## 例

:label: `subsec_probability_hiv_app`

自分のスキルを試してみましょう。医師が患者に HIV 検査を実施すると仮定します。この検査はかなり正確で、患者が健康であるにもかかわらず病気であると報告した場合、検査が失敗する確率はわずか 1% です。さらに、患者が実際に HIV に感染している場合には、必ず HIV を検出します。 $D_1 \in {0, 1}$ を使用して診断を示し (陰性の場合は $0$、陽性の場合は $1$)、HIV ステータスを示すのに $H \in {0, 1}$ を使用します。

 |条件付き確率 | $H=1$ | $H=0$ | |:----------------------|------:|------:| | $P(D_1 = 1 \mid H)$ | 1 | 0.01 | | $P(D_1 = 0 \mid H)$ | 0 | 0.99 |

列の合計は条件付き確率であるため、すべて 1 であることに注意してください (ただし、行の合計はそうではありません)。検査で陽性反応が出た場合に患者が HIV に感染している確率、つまり $P(H = 1 \mid D_1 = 1)$ を計算してみましょう。直観的には、この病気の頻度は誤報の数に影響するため、病気の頻度によって決まります。母集団がかなり健全であると仮定します (例: $P(H=1) = 0.0015$)。ベイズの定理を適用するには、限界化を適用して決定する必要があります。

 $$\begin{aligned} P(D_1 = 1) =&amp; P(D_1=1, H=0) + P(D_1=1, H=1) \ =&amp; P(D_1=1 \mid H=0) P(H=0) + P(D_1=1 \mid H=1) P(H=1) \ =&amp; 0.011485。 \end{整列} $$

これは私たちに次のことをもたらします

$$P(H = 1 \mid D_1 = 1) = \frac{P(D_1=1 \mid H=1) P(H=1)}{P(D_1=1)} = 0.1306.$$

言い換えれば、非常に正確な検査を使用したにもかかわらず、患者が実際に HIV に感染している可能性は 13.06% しかありません。ご覧のとおり、確率は直感に反する場合があります。このような恐ろしい知らせを聞いた患者はどうすべきでしょうか？おそらく、患者は医師に、明確にするために別の検査を実施するよう依頼するでしょう。 2 番目のテストには異なる特性があり、最初のテストほど良くはありません。

 |条件付き確率 | $H=1$ | $H=0$ | |:----------------------|------:|------:| | $P(D_2 = 1 \mid H)$ | 0.98 | 0.03 | | $P(D_2 = 0 \mid H)$ | 0.02 | 0.97 |

残念ながら、2回目の検査でも陽性反応が出ました。条件付き独立性を仮定して、ベイズの定理を呼び出すために必要な確率を計算してみましょう。

 $$\begin{aligned} P(D_1 = 1, D_2 = 1 \mid H = 0) &amp; = P(D_1 = 1 \mid H = 0) P(D_2 = 1 \mid H = 0) =&amp; 0.0003, \ P(D_1 = 1, D_2 = 1 \mid H = 1) &amp; = P(D_1 = 1 \mid H = 1) P(D_2 = 1 \mid H = 1) =&amp; 0.98。 \end{整列} $$

ここで、周縁化を適用して、両方のテストが陽性となる確率を取得できます。

 $$\begin{aligned} &amp;P(D_1 = 1, D_2 = 1)\ =&amp; P(D_1 = 1, D_2 = 1, H = 0) + P(D_1 = 1, D_2 = 1, H = 1) \ =&amp; P(D_1 = 1、D_2 = 1 \mid H = 0)P(H=0) + P(D_1 = 1、D_2 = 1 \mid H = 1)P(H=1)\ =&amp; 0.00176955。 \end{整列} $$

最後に、両方の検査が陽性である場合に、患者が HIV に感染している確率は次のようになります。

 $$P(H = 1 \mid D_1 = 1, D_2 = 1) = \frac{P(D_1 = 1, D_2 = 1 \mid H=1) P(H=1)}{P(D_1 = 1, D_2 = 1)} = 0.8307.$$

つまり、2 回目のテストにより、すべてがうまくいっていないという確信がさらに高まりました。 2 回目のテストは最初のテストよりも精度が大幅に低下しましたが、それでも推定値は大幅に改善されました。両方のテストが条件付きで互いに独立しているという仮定は、より正確な推定値を生成するために重要でした。同じテストを 2 回実行するという極端なケースを考えてみましょう。この状況では、両方の時間で同じ結果が期待されるため、同じテストを再度実行しても追加の洞察は得られません。賢明な読者は、診断が、より多くの特徴 (検査結果) を取得するにつれて、患者が健康であるかどうかを判断する能力が向上する、目に見えないところに隠れている分類器のように動作することに気づいたかもしれません。

## 期待

多くの場合、意思決定を行うには、個々のイベントに割り当てられた確率を調べるだけでなく、それらを組み合わせて、指針を提供できる有用な集計を行う必要があります。たとえば、確率変数が連続スカラー値を取る場合、*平均して*どのような値が期待されるかを知ることがよく重要になります。この量は正式には*期待値*と呼ばれます。私たちが投資を行っている場合、最初の利息の量は、考えられるすべての結果を平均した (および適切な確率で重み付けされた) 期待できるリターンである可能性があります。たとえば、50% の確率で投資が完全に失敗する可能性があり、40% の確率で 2$\times$ のリターンが得られる可能性があり、10% の確率で 10$\times$ のリターンが得られる可能性があるとします。 $。期待収益を計算するには、すべての収益を合計し、それぞれに収益が発生する確率を掛けます。これにより、期待値 $0.5 \cdot 0 + 0.4 \cdot 2 + 0.1 \cdot 10 = 1.8$ が得られます。したがって、期待収益は 1.8$\times$ となります。

一般に、確率変数 $X$ の*期待値*(または平均) は次のように定義されます。

 $$E[X] = E_{x \sim P}[x] = \sum_{x} x P(X = x).$$

同様に、密度については $E[X] = \int x ;dp(x)$ を取得します。 $x$ の関数の期待値に興味がある場合があります。これらの期待値は次のように計算できます。

 $$E_{x \sim P}[f(x)] = \sum_x f(x) P(x) \text{ および } E_{x \sim P}[f(x)] = \int f(x ) p(x) ;dx$$

それぞれ離散確率と密度の場合。上記の投資の例に戻ると、$f$ は収益に関連付けられた*効用*(幸福) である可能性があります。行動経済学者たちは、人々はベースラインと比較して1ドルを稼ぐことで得られる効用よりも、より大きな不効用をお金を失うことと結びつけると指摘してきた。さらに、お金の価値は非線形になる傾向があります。 10万ドルを所有するのとゼロドルを所有するのとでは、家賃を払い、よく食べ、質の高い医療を享受できるか、ホームレスになって苦しむかの違いが生じます。一方で、100k を所有する場合と 200k を所有する場合の利益はそれほど劇的ではありません。このような推論は、「お金の効用は対数である」という決まり文句の動機になります。

合計損失に伴う効用が -1 で、収益に伴う効用がそれぞれ 1、2、10 であった場合、投資によって期待される幸福度は $0.5 \cdot (-1) + となります。 0.4 \cdot 2 + 0.1 \cdot 4 = 0.7$ (期待される効用の損失は 30%)。実際にこれがあなたの効用関数であるならば、お金を銀行に預けておくのが最善かもしれません。

財務上の決定については、投資がどの程度*リスクが*あるかを測定することも必要になる場合があります。ここでは、期待値だけでなく、実際の値がこの値に対してどの程度*変動する*傾向があるかにも注目します。実際の値と期待値の差を単に期待することはできないことに注意してください。それは、差の期待値は期待値の差であるため、$E[X - E[X]] = E[X] - E[E[X]] = 0$ となるからです。ただし、この差の非負関数の期待値を調べることができます。確率変数の*分散は*、*二乗*偏差の期待値を調べることによって計算されます。

 $$\mathrm{Var}[X] = E\left[(X - E[X])^2\right] = E[X^2] - E[X]^2.$$

ここで、 $(X - E[X])^2 = X^2 - 2 XE[X] + E[X]^2$ を展開し、各項の期待値を取得することで等式が得られます。分散の平方根は、*標準偏差*と呼ばれるもう 1 つの有用な量です。分散と標準偏差は同じ情報を伝えますが (どちらも他方から計算できます)、標準偏差には、確率変数で表される元の量と同じ単位で表されるという優れた特性があります。

最後に、確率変数の関数の分散は、同様に次のように定義されます。

 $$\mathrm{Var} *{x \sim P}[f(x)] = E* {x \sim P}[f^2(x)] - E_{x \sim P}[f(x)]^ 2.$$

投資の例に戻ると、投資の分散を計算できます。 $0.5 \cdot 0 + 0.4 \cdot 2^2 + 0.1 \cdot 10^2 - 1.8^2 = 8.36$ で求められます。どう考えても、これはリスクの高い投資です。数学的な慣例により、平均と分散は $\mu$ および $\sigma^2$ として参照されることが多いことに注意してください。これは、ガウス分布をパラメータ化するために使用する場合に特によく見られます。

*スカラー*確率変数の期待値と分散を導入したのと同じ方法で、ベクトル値の確率変数に対してもそれを行うことができます。要素ごとに適用できるため、期待は簡単です。たとえば、 $\boldsymbol{\mu} \stackrel{\mathrm{def}}{=} E_{\mathbf{x} \sim P}[\mathbf{x}]$ の座標は $\mu_i = E_{\ mathbf{x} \sim P}[x_i]$。共分散はさらに複雑です。この問題は、確率変数とその平均の差の*外積*の期待値を取得することで解決します。

 $$\boldsymbol{\Sigma} \stackrel{\mathrm{def}}{=} \mathrm{Cov} *{\mathbf{x} \sim P}[\mathbf{x}] = E* {\mathbf{x} \sim P}\left[(\mathbf{x} - \boldsymbol{\mu}) (\mathbf{x} - \boldsymbol{\mu})^\top\right].$$

この行列 $\boldsymbol{\Sigma}$ は共分散行列と呼ばれます。その効果を確認する簡単な方法は、$\mathbf{x}$ と同じサイズのベクトル $\mathbf{v}$ を考慮することです。したがって、

 $$\mathbf{v}^\top \boldsymbol{\Sigma} \mathbf{v} = E_{\mathbf{x} \sim P}\left[\mathbf{v}^\top(\mathbf{x} - \boldsymbol{\mu}) (\mathbf{x} - \boldsymbol{\mu})^\top \mathbf{v}\right] = \mathrm{Var}_{x \sim P}[\mathbf{ v}^\top \mathbf{x}].$$

そのため、$\boldsymbol{\Sigma}$ を使用すると、単純な行列の乗算によって $\mathbf{x}$ の線形関数の分散を計算できます。非対角要素は、座標の相関関係を示します。値 0 は相関がないことを意味し、正の値が大きいほど相関が強いことを意味します。



## 議論

機械学習には不明な点がたくさんあります。入力が与えられた場合、ラベルの値が不確かになることがあります。パラメータの推定値が不確実な場合があります。デプロイメント時に到着するデータがトレーニング データと同じ分布からのものであるかどうかさえ不確かになる可能性があります。

*偶然の不確実性*とは、問題に固有であり、観測変数では説明できない真のランダム性に起因する不確実性を示します。 *「認識論的不確実性」*とは、モデルのパラメータに関する不確実性、つまりより多くのデータを収集することで削減できると期待できる種類の不確実性を示します。コインが表になる確率については認識論的な不確実性があるかもしれませんが、この確率がわかったとしても、将来のトスの結果については偶然的な不確実性が残ります。誰かが公平なコインを投げるのをどれだけ長く見ていたとしても、次のコインを投げたときに表になるという確信は 50% 以上もそれ以下もありません。これらの用語は、機械モデリングの文献によるものです ([不確実性の定量](https://en.wikipedia.org/wiki/Uncertainty_quantification)化のこの側面に関するレビューについては、:citet: `Der-Kiureghian.Ditlevsen.2009`などを参照)。これらの用語は軽度の言葉の乱用に当たることに注意してください。*認識論*という用語は*知識*に関するあらゆるものを指します。したがって、哲学的な意味では、すべての不確実性は認識論的です。

未知の確率分布からのサンプリング データにより、データ生成分布のパラメーターを推定するために使用できる情報が得られることがわかりました。とはいえ、これが可能になる速度は非常に遅い可能性があります。私たちのコイン投げの例 (および他の多くの例) では、$1/\sqrt{n}$ のレートで収束する推定量を設計する以外に何もできません。ここで、$n$ はサンプル サイズ (例: 投げた回数) です。 。これは、観測値が 10 から 1000 に変更されると (通常は非常に達成可能なタスクです)、不確実性が 10 分の 1 に減少するのに対し、次の 1000 観測ではあまり役に立たず、わずか 1.41 倍の減少しか得られないことを意味します。これは機械学習の永続的な特徴です。多くの場合、簡単に利益が得られますが、さらに利益を得るには非常に大量のデータが必要となり、それに伴い膨大な量の計算が必要になることがよくあります。大規模言語モデルに関するこの事実の実証的レビューについては、:citet: `Revels.Lubin.Papamarkou.2016`を参照してください。

また、統計モデリング用の言語とツールも強化しました。その過程で、条件付き確率と、統計学で最も重要な方程式の 1 つであるベイズの定理について学びました。これは、観測値 $B$ がパラメーター $A$ と事前確率 $P(A) の選択にどの程度一致するかを扱う尤度項 $P(B \mid A)$ を通じて、データによって伝えられる情報を分離する効果的なツールです。 $ は、そもそも $A$ の特定の選択がどの程度妥当であるかを決定します。特に、このルールを、検査の有効性*と*病気自体の有病率 (つまり、事前の) に基づいて、診断に確率を割り当てるためにどのように適用できるかを確認しました。

最後に、特定の確率分布、つまり期待値と分散の影響に関する重要な質問の最初のセットを導入しました。確率分布には線形および二次の期待以外にも多くの期待が存在しますが、これら 2 つは分布の起こり得る動作についてすでにかなりの知識を提供しています。たとえば、[チェビシェフの不等式](https://en.wikipedia.org/wiki/Chebyshev%27s_inequality)は $P(|X - \mu| \geq k \sigma) \leq 1/k^2$ を示します。ここで、$\mu$ は期待値、$\sigma^2$ は分散です。 $k &gt; 1$ は私たちが選択した信頼パラメータです。これは、分布からの引き出しは、期待値を中心とした $[-\sqrt{2} \sigma, \sqrt{2} \sigma]$ 区間内に少なくとも 50% の確率で存在することを示しています。

## 演習
1. より多くのデータを観察することで、結果に関する不確実性の量を任意の低いレベルまで減らすことができる例を挙げてください。
1. より多くのデータを観察しても、ある点までは不確実性の量が減少するだけで、それ以上は減少しないという例を挙げてください。これが当てはまる理由と、この点がどこで発生すると予想されるかを説明してください。
1. 私たちは、コイントスの平均値への収束を経験的に実証しました。 $n$ 個のサンプルを抽出した後に表が見える確率の推定値の分散を計算します。<ol><li>分散は観測値の数に応じてどのように変化するのでしょうか?
1. チェビシェフの不等式を使用して、期待からの逸脱を制限します。
1. それは中心極限定理とどのように関係するのでしょうか?
1. これまでに得た情報に基づいて、$H=0$ と仮定して、$D_1$ と $D_2$ の同時確率テーブルを計算します。
1.  1 回の検査で陽性が得られた後、患者が陽性になる確率 ($H=1$) を導き出します。以前と同じベースライン確率 $P(H=1) = 0.0015$ を仮定できます。
1. 両方の検査で陽性が得られた後、患者が陽性になる確率 ($H=1$) を導き出します。
1. 特定のポートフォリオ $\boldsymbol{\alpha}$ の期待収益を計算します。
1. ポートフォリオの収益を最大化したい場合、どのように投資を選択すればよいでしょうか?
1. ポートフォリオの*分散を*計算します。
1. 分散を上限に制限しながら収益を最大化する最適化問題を定式化します。これはノーベル賞を受賞した[マルコヴィッツのポートフォリオ](https://en.wikipedia.org/wiki/Markowitz_model)です:cite: `Mangram.2013` 。これを解決するには、二次計画法ソルバーが必要になりますが、これは本書の範囲をはるかに超えています。



[ディスカッション](https://discuss.d2l.ai/t/37)
